Three text embeddings:

Model 1: Spacy small svd 40

Model 2: TFIDF svd 5

Model 3: FastText svd 42

Use these three (without tabular features) models, as first-stage models.

Put the 3 different predicted deal_probability, together with tabular features, as final meta-model to predict the final deal_probability.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import lightgbm as lgb
from sklearn.decomposition import TruncatedSVD

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir_path = '/content/drive/MyDrive/ML Project/data/'
spacy_small_train = pd.read_csv(data_dir_path + 'train_embeddings_spacysm.csv')
spacy_small_test = pd.read_csv(data_dir_path + 'test_embeddings_spacysm.csv')
spacy_small_test.head()

,item_id,user_id,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,...,embedding_86,embedding_87,embedding_88,embedding_89,embedding_90,embedding_91,embedding_92,embedding_93,embedding_94,embedding_95
0,bd45a493569d,6be13830198e,0.540213,0.471374,0.378489,0.294629,0.147339,1.191240,0.368049,-0.299011,...,-0.267030,0.253568,-1.319817,0.191024,-0.262129,0.170887,0.296282,0.758039,0.248311,-0.374299
1,321127a6a33b,98b93e9aaf33,0.384541,0.197485,0.754817,-0.113432,0.561113,-0.447124,0.114448,0.256563,...,-0.134967,0.367342,-0.236856,0.497181,-0.042024,0.176952,0.494697,0.530671,-0.430334,-0.149922
2,d976d5211526,7246f2b892e9,-0.029891,0.407801,0.318676,0.582881,0.244637,-0.359194,0.477885,-0.194458,...,-0.569900,0.522779,-0.603296,0.185623,0.200511,0.299057,-0.104154,0.571752,-0.410092,-0.982872
3,c046d8685345,3ceed1f36bec,0.335886,0.011270,0.380748,-0.211154,0.616863,-0.120722,0.310040,0.499794,...,-0.206480,0.250499,-0.291200,0.350235,-0.041513,0.368104,0.227451,0.572295,-0.285416,0.130250
4,98a8ba1ff4c4,feb28eeb11a9,-0.171509,0.082136,0.366043,0.891841,0.874531,0.291206,0.344731,-0.543677,...,-0.625143,0.228471,-1.239442,-0.155060,0.222965,0.481061,-0.238026,0.854457,0.211418,-0.051000


In [ ]:
spacy_small_train

,item_id,user_id,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,...,embedding_86,embedding_87,embedding_88,embedding_89,embedding_90,embedding_91,embedding_92,embedding_93,embedding_94,embedding_95
0,771fda0ff8ff,969f1c4acd66,-0.784817,1.128159,-0.194680,0.201708,-0.032032,0.997670,-0.029699,-0.420449,...,0.268015,0.361722,-0.211765,-0.254548,0.391499,0.534850,-0.303226,-0.269790,-0.999904,-0.171014
1,3c296fc97db9,cf345facfe6b,0.338688,0.004488,-0.134089,0.583048,0.919529,0.604952,0.474413,-0.201748,...,-0.354492,0.343257,-0.541383,-0.085286,-0.305382,0.395390,-0.213868,0.618845,-0.674519,-0.192896
2,13ecb01f22c1,a4ad6dcc8446,0.294649,0.409093,0.873846,-0.124767,0.613569,-0.644641,-0.331209,0.229863,...,-0.518137,0.363129,-0.633150,0.538680,-0.108103,-0.646061,0.281124,0.572991,-0.601334,0.102266
3,f3a81005ce3d,a41d7a0cd76f,0.452859,0.550829,0.164094,0.400878,0.168170,-0.225049,0.936240,-0.064317,...,-0.332345,1.708878,-0.514540,0.189822,0.618482,0.846577,0.711320,0.275401,-0.804954,-0.228114
4,bc91d5eb9229,19757fb0047c,0.752452,0.679213,1.093276,-0.732083,1.000517,-0.581267,-0.047663,0.207912,...,-0.164582,0.467063,-0.618151,0.877890,-0.100737,0.348708,0.527191,0.410933,-0.166898,-0.286146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202734,f09ae5daaf01,f9b5910f655a,0.092952,0.519809,-0.223772,-0.195881,0.440328,0.270696,0.111657,-0.215432,...,-0.162616,0.522815,-0.504649,0.830579,-0.212667,0.738441,0.128696,0.648362,-0.463473,0.072351
1202735,6eab9f61133f,1decc320fb12,0.318112,0.556246,0.538502,1.015098,0.267589,0.847872,0.410118,0.098327,...,-0.459123,0.052585,-0.689799,0.190541,-0.075423,0.110757,0.442768,0.933104,-0.184098,-0.472108
1202736,9b5c1f4fca75,196588f9b315,0.420332,0.067659,-0.123687,0.116546,0.260956,0.317893,0.517853,0.382519,...,0.023890,0.116941,-0.377538,0.267939,0.045237,0.214653,0.201046,0.398684,-0.532617,-0.437664
1202737,22c8381d9eab,39b04d8ac30a,-0.010366,0.318798,0.250792,-0.087531,0.182344,0.557165,0.139546,-0.274614,...,0.074618,0.131527,-0.487200,0.018191,-0.148018,0.324981,0.236242,0.278023,-0.404480,-0.415926


Spacy_small reduce svd 40

In [ ]:
embedding_columns = [col for col in spacy_small_train.columns if col.startswith('embedding_')]


svd = TruncatedSVD(n_components=40, random_state=42)


train_embeddings = spacy_small_train[embedding_columns].values
train_svd_embeddings = svd.fit_transform(train_embeddings)

test_embeddings = spacy_small_test[embedding_columns].values
test_svd_embeddings = svd.transform(test_embeddings)

train_svd_df = pd.DataFrame(train_svd_embeddings, columns=[f'spacy_svd_{i}' for i in range(40)])
test_svd_df = pd.DataFrame(test_svd_embeddings, columns=[f'spacy_svd_{i}' for i in range(40)])

train_spacy_svd40 = pd.concat([spacy_small_train.drop(columns=embedding_columns), train_svd_df], axis=1)
test_spacy_svd40 = pd.concat([spacy_small_test.drop(columns=embedding_columns), test_svd_df], axis=1)

train_spacy_svd40.head()


,item_id,user_id,spacy_svd_0,spacy_svd_1,spacy_svd_2,spacy_svd_3,spacy_svd_4,spacy_svd_5,spacy_svd_6,spacy_svd_7,...,spacy_svd_30,spacy_svd_31,spacy_svd_32,spacy_svd_33,spacy_svd_34,spacy_svd_35,spacy_svd_36,spacy_svd_37,spacy_svd_38,spacy_svd_39
0,771fda0ff8ff,969f1c4acd66,-2.522808,2.116002,-0.869886,0.237252,1.284821,-2.616579,0.893342,-0.683051,...,0.070859,-0.200402,0.399969,0.464117,-0.783324,0.042151,0.461139,0.033606,0.273473,0.345149
1,3c296fc97db9,cf345facfe6b,-3.110800,0.421362,-0.908467,1.224478,-0.552887,-0.032324,1.927485,-1.136356,...,-0.460100,0.109586,-0.204724,-0.232934,-0.253469,0.013493,0.305173,0.093155,0.241529,0.004352
2,13ecb01f22c1,a4ad6dcc8446,-2.976814,-2.834591,0.235023,-0.270363,-0.188095,0.239168,0.069469,0.283459,...,-0.009834,-0.376687,0.270234,0.186656,-0.457154,0.029784,-0.187105,-0.593824,-0.213089,0.623080
3,f3a81005ce3d,a41d7a0cd76f,-3.214114,-1.702089,-0.787871,-0.591761,1.056741,-1.872029,-1.164354,1.378160,...,-0.146123,0.093856,-0.400256,-0.540356,0.629810,-0.108631,-0.443335,0.297444,0.112421,0.046438
4,bc91d5eb9229,19757fb0047c,-2.796490,-2.689414,0.306709,-0.054795,-0.514153,0.651754,-0.203986,-0.074676,...,0.334175,0.183384,-0.140712,0.173028,-0.004446,0.184789,-0.004080,0.102282,0.066628,0.176290


In [ ]:
spacy_small_train.shape

(1202739, 98)

In [ ]:
tfidf_svd5_train[['item_id','user_id']]

,item_id,user_id
0,771fda0ff8ff,969f1c4acd66
1,3c296fc97db9,cf345facfe6b
2,13ecb01f22c1,a4ad6dcc8446
3,f3a81005ce3d,a41d7a0cd76f
4,bc91d5eb9229,19757fb0047c
...,...,...
1202734,f09ae5daaf01,f9b5910f655a
1202735,6eab9f61133f,1decc320fb12
1202736,9b5c1f4fca75,196588f9b315
1202737,22c8381d9eab,39b04d8ac30a


In [ ]:
tfidf_svd5_train = pd.read_csv(data_dir_path + 'train_embeddings_tfidf_svd.csv')
tfidf_svd5_test = pd.read_csv(data_dir_path + 'test_embeddings_tfidf_svd.csv')
tfidf_svd5_train


,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,user_type,region_city,...,item_seq_number,activation_date,image,image_top_1,deal_probability,svd_0,svd_1,svd_2,svd_3,svd_4
0,Омская область,Омск,Бытовая электроника,Настольные компьютеры,NaN,NaN,NaN,50.0,Private,Омская область_Омск,...,18,2017-03-28,5f1a4dc40d333d5cd20c9831553a61053e83ffb156f692...,2789.0,0.76786,0.426458,0.469653,-0.034854,-0.188869,0.071247
1,Башкортостан,Уфа,Хобби и отдых,Спорт и отдых,Зимние виды спорта,NaN,NaN,500.0,Private,Башкортостан_Уфа,...,30,2017-03-16,659f553a15733bbc034bc2ea321a8aaa39feaa3fb1e410...,2645.0,0.00000,0.205034,0.184058,-0.002020,-0.046927,0.010675
2,Нижегородская область,Дзержинск,Для дома и дачи,Растения,NaN,NaN,NaN,80.0,Private,Нижегородская область_Дзержинск,...,25,2017-03-16,7f5b347e054800f384d22a9959792ab6da7b1442931d03...,406.0,0.12311,0.059681,-0.012399,0.014739,0.088000,-0.040258
3,Челябинская область,Миасс,Личные вещи,Детская одежда и обувь,Для девочек,Верхняя одежда,98-104 см (2-4 года),1500.0,Private,Челябинская область_Миасс,...,182,2017-03-26,f17dae532061e4a29e25a44fbd90d40ae894c697d636d5...,94.0,0.80323,0.226856,-0.037198,-0.069922,-0.064799,-0.013395
4,Башкортостан,Уфа,Для дома и дачи,Бытовая техника,Для кухни,Плиты,NaN,18990.0,Shop,Башкортостан_Уфа,...,422,2017-03-16,ef7789caae6b81233991c9485cf081bcd6530aa4bd9735...,1852.0,0.00000,0.108313,-0.029808,0.028333,0.140245,-0.118581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202734,Ростовская область,Ростов-на-Дону,Услуги,Предложение услуг,"Ремонт, строительство",Сантехника,NaN,NaN,Private,Ростовская область_Ростов-на-Дону,...,5,2017-03-21,NaN,NaN,0.20000,0.014492,0.000669,0.002342,0.044897,0.021966
1202735,Тульская область,Тула,Недвижимость,Квартиры,Сдам,На длительный срок,2,13500.0,Company,Тульская область_Тула,...,1211,2017-03-17,19febf47e8ef31b14679e19f276268f6120a0f25b062c8...,2220.0,0.03703,0.079886,-0.004673,-0.007415,0.316639,0.303809
1202736,Белгородская область,Алексеевка,Услуги,Предложение услуг,Другое,NaN,NaN,700.0,Private,Белгородская область_Алексеевка,...,5,2017-03-17,bbee5106cfd831a64aa784f946857088c6cc935333faac...,148.0,0.20000,0.067934,-0.019119,-0.003423,0.060296,-0.001410
1202737,Челябинская область,Челябинск,Личные вещи,Товары для детей и игрушки,Детские коляски,NaN,NaN,9000.0,Private,Челябинская область_Челябинск,...,21,2017-03-15,23d6b6b7d9903bc236af71dc5abbe93762b5382f4062e8...,1002.0,0.80323,0.201997,-0.043083,-0.056080,-0.016665,0.006040


In [ ]:
svd_columns = [col for col in tfidf_svd5_train.columns if col.startswith('svd_')]

new_column_names = {col: f"tfidf_{col}" for col in svd_columns}

tfidf_svd5_train.rename(columns=new_column_names, inplace=True)
tfidf_svd5_test.rename(columns=new_column_names, inplace=True)
tfidf_svd5_test

,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,user_type,title,...,activation_date,image,image_top_1,deal_probability,text,tfidf_svd_0,tfidf_svd_1,tfidf_svd_2,tfidf_svd_3,tfidf_svd_4
0,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Аксессуары,NaN,NaN,50000.0,Private,Сумка бренд Louis Vuitton,...,2017-03-19,3e6502f1657e3f256be696fd82feb455ddc82345e3577e...,256.0,0.00000,"Сумка бренд Louis Vuitton Сумка коллекционная,...",0.183250,-0.150625,-0.140535,-0.037772,-0.014746
1,Ярославская область,Ярославль,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,21950.0,Company,Автокресло Carmate (Ailebebe) Swing Moon Beige,...,2017-03-18,9270dff29b16082e5a01d35a418cd6d525b6618c33fc58...,796.0,0.00000,Автокресло Carmate (Ailebebe) Swing Moon Beige...,0.102642,-0.026940,0.000830,0.131964,-0.048938
2,Тюменская область,Тюмень,Личные вещи,"Одежда, обувь, аксессуары",Мужская одежда,Пиджаки и костюмы,52–54 (XXL),2500.0,Private,Пиджак кожанный,...,2017-03-19,bbe390ab0a1ed2cb97c4ed064524e9809825dfeba713c0...,655.0,0.00000,"Пиджак кожанный Состояние на 4, торг",0.186076,-0.018780,0.054649,0.231072,-0.195388
3,Новосибирская область,Новосибирск,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,32,750.0,Company,Сапоги на слякоть котофей нов,...,2017-03-20,e528c5df69dbc8d6073e9b617b174bec2405cf7e6e3689...,91.0,0.00000,Сапоги на слякоть котофей нов Сапожки из невес...,0.063535,-0.010225,0.000436,0.074031,-0.051945
4,Краснодарский край,Новороссийск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,42–44 (S),200.0,Private,Юбка,...,2017-03-19,ce15238bc8d1d093b79ade25f1346be1acfc7d9570ec1b...,81.0,0.00000,"Юбка Юбка женская, размер 42-44, цвет черный, ...",0.194431,-0.157453,-0.049707,0.006616,-0.085714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300680,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Обувь,37,1000.0,Private,Сабо,...,2017-03-17,3d1962234a5f72ea12dae871d7af1f7f5016094ef92a9d...,445.0,0.00000,"Сабо Состояние хорошее,замша натуральная.",0.112901,0.004502,0.073649,0.198366,-0.265673
300681,Свердловская область,Екатеринбург,Бытовая электроника,Телефоны,LG,NaN,NaN,5500.0,Private,Lg Leon h 324,...,2017-03-26,18210297b0e61f0aed44722dd7a2e2920cd9b8ee404e47...,2942.0,0.07122,Lg Leon h 324 Документы имеются.В хорошем рабо...,0.195719,-0.158146,-0.159542,-0.053266,0.051253
300682,Тульская область,Тула,Личные вещи,Товары для детей и игрушки,Товары для купания,NaN,NaN,150.0,Private,Ванна детская,...,2017-03-25,7542a8ce604ab56601027660a4f181d66d642e8627a492...,1019.0,0.00000,Ванна детская Ванночка детская. Цвет Лайма,0.041695,-0.015667,-0.000320,0.043090,-0.012339
300683,Свердловская область,Екатеринбург,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,24,300.0,Private,"Сапоги эва, размер 24-25",...,2017-03-18,6b08256fcd0c6f81760b97c8b5a02f711919870333d9b7...,89.0,0.80323,"Сапоги эва, размер 24-25 Внутри съемный носок",0.087364,-0.032391,0.023202,0.085719,-0.128796


In [ ]:
tfidf_svd5_train.columns

Index(['region', 'city', 'parent_category_name', 'category_name', 'param_1',
       'param_2', 'param_3', 'price', 'user_type', 'region_city',
       'all_category', 'category_param_1', 'region_category_user',
       'city_category_user', 'category_price_mean', 'category_price_std',
       'category_price_skew', 'city_price_mean', 'city_price_max',
       'city_price_skew', 'title_length', 'description_length',
       'title_word_count', 'description_word_count', 'title_has_keyword',
       'description_has_keyword', 'title_digit_count',
       'description_digit_count', 'description_newline_count', 'price_log',
       'price_bin', 'price_to_category_mean', 'price_to_category_max',
       'description_missing', 'item_id', 'user_id', 'item_seq_number',
       'activation_date', 'image', 'image_top_1', 'deal_probability', 'svd_0',
       'svd_1', 'svd_2', 'svd_3', 'svd_4'],
      dtype='object')

In [ ]:
fasttext_svd42_train = pd.read_csv(data_dir_path + 'train_embeddings_fasttext_svd.csv')
fasttext_svd42_test = pd.read_csv(data_dir_path + 'test_embeddings_fasttext_svd.csv')
fasttext_svd42_train

,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,user_type,region_city,...,svd_32,svd_33,svd_34,svd_35,svd_36,svd_37,svd_38,svd_39,svd_40,svd_41
0,Омская область,Омск,Бытовая электроника,Настольные компьютеры,NaN,NaN,NaN,50.0,Private,Омская область_Омск,...,-0.590956,-1.087211,-0.799874,-1.122621,1.701886,-0.252314,1.493885,-0.257374,-1.266095,0.891860
1,Башкортостан,Уфа,Хобби и отдых,Спорт и отдых,Зимние виды спорта,NaN,NaN,500.0,Private,Башкортостан_Уфа,...,0.172983,-0.187748,-0.891440,0.491830,-0.566420,-0.542912,-0.598151,-0.369308,-0.956708,0.502890
2,Нижегородская область,Дзержинск,Для дома и дачи,Растения,NaN,NaN,NaN,80.0,Private,Нижегородская область_Дзержинск,...,-1.212741,0.183845,0.383880,0.432686,-0.552630,0.748625,-0.447899,-0.383560,-0.415854,-0.177809
3,Челябинская область,Миасс,Личные вещи,Детская одежда и обувь,Для девочек,Верхняя одежда,98-104 см (2-4 года),1500.0,Private,Челябинская область_Миасс,...,-0.095292,-0.399213,-1.308965,0.644886,1.579159,-0.561029,-0.379097,0.323405,0.223288,0.062198
4,Башкортостан,Уфа,Для дома и дачи,Бытовая техника,Для кухни,Плиты,NaN,18990.0,Shop,Башкортостан_Уфа,...,-0.235615,-0.602194,0.059004,1.343861,-0.908024,0.582427,-0.212719,0.209566,-0.100444,0.424449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202734,Ростовская область,Ростов-на-Дону,Услуги,Предложение услуг,"Ремонт, строительство",Сантехника,NaN,NaN,Private,Ростовская область_Ростов-на-Дону,...,6.839081,0.687795,-3.202302,-0.178912,-0.104948,-0.515421,1.246996,0.772102,-1.687494,1.114167
1202735,Тульская область,Тула,Недвижимость,Квартиры,Сдам,На длительный срок,2,13500.0,Company,Тульская область_Тула,...,0.844168,0.034772,0.269135,-0.155830,0.859433,0.549835,-0.399183,0.399499,0.227667,-0.012890
1202736,Белгородская область,Алексеевка,Услуги,Предложение услуг,Другое,NaN,NaN,700.0,Private,Белгородская область_Алексеевка,...,-0.302642,1.104279,-0.890172,1.039392,0.433664,-0.424274,-0.754427,-1.486754,0.211968,-0.159443
1202737,Челябинская область,Челябинск,Личные вещи,Товары для детей и игрушки,Детские коляски,NaN,NaN,9000.0,Private,Челябинская область_Челябинск,...,0.478689,-0.994209,1.299318,-0.486527,1.627801,-1.688751,1.098206,0.748161,0.059967,0.246059


In [ ]:
columns_to_keep = ['item_id', 'user_id'] + [col for col in fasttext_svd42_train.columns if col.startswith('svd_')]

fasttext_svd42_train_filtered = fasttext_svd42_train[columns_to_keep]
fasttext_svd42_test_filtered = fasttext_svd42_test[columns_to_keep]

svd_columns = [col for col in fasttext_svd42_train_filtered.columns if col.startswith('svd_')]
new_svd_column_names = {col: f"fasttext_{col}" for col in svd_columns}

fasttext_svd42_train_filtered.rename(columns=new_svd_column_names, inplace=True)
fasttext_svd42_test_filtered.rename(columns=new_svd_column_names, inplace=True)

fasttext_svd42_train_filtered

,item_id,user_id,fasttext_svd_0,fasttext_svd_1,fasttext_svd_2,fasttext_svd_3,fasttext_svd_4,fasttext_svd_5,fasttext_svd_6,fasttext_svd_7,...,fasttext_svd_32,fasttext_svd_33,fasttext_svd_34,fasttext_svd_35,fasttext_svd_36,fasttext_svd_37,fasttext_svd_38,fasttext_svd_39,fasttext_svd_40,fasttext_svd_41
0,771fda0ff8ff,969f1c4acd66,18.712120,1.665349,-0.316207,-0.865870,1.004217,-4.422844,-1.784763,0.819020,...,-0.590956,-1.087211,-0.799874,-1.122621,1.701886,-0.252314,1.493885,-0.257374,-1.266095,0.891860
1,3c296fc97db9,cf345facfe6b,17.063452,-5.314239,-0.158923,0.593008,1.314238,-1.136343,2.142924,0.214027,...,0.172983,-0.187748,-0.891440,0.491830,-0.566420,-0.542912,-0.598151,-0.369308,-0.956708,0.502890
2,13ecb01f22c1,a4ad6dcc8446,8.301052,1.407741,-1.524526,4.494484,-0.058221,-0.944867,0.073577,0.157553,...,-1.212741,0.183845,0.383880,0.432686,-0.552630,0.748625,-0.447899,-0.383560,-0.415854,-0.177809
3,f3a81005ce3d,a41d7a0cd76f,13.366659,-0.316903,4.525409,-2.305178,-0.898954,2.448828,-1.329531,1.810665,...,-0.095292,-0.399213,-1.308965,0.644886,1.579159,-0.561029,-0.379097,0.323405,0.223288,0.062198
4,bc91d5eb9229,19757fb0047c,13.153263,5.237125,-10.867508,0.831236,-3.664208,2.993702,1.709104,-0.997776,...,-0.235615,-0.602194,0.059004,1.343861,-0.908024,0.582427,-0.212719,0.209566,-0.100444,0.424449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202734,f09ae5daaf01,f9b5910f655a,11.927881,6.155726,-3.977766,2.949712,2.087063,-2.669618,3.016116,2.066788,...,6.839081,0.687795,-3.202302,-0.178912,-0.104948,-0.515421,1.246996,0.772102,-1.687494,1.114167
1202735,6eab9f61133f,1decc320fb12,13.597333,5.843094,0.717789,-6.002838,5.886590,-0.057484,1.508186,1.862631,...,0.844168,0.034772,0.269135,-0.155830,0.859433,0.549835,-0.399183,0.399499,0.227667,-0.012890
1202736,9b5c1f4fca75,196588f9b315,12.830471,7.033854,-2.242913,6.332954,1.214505,-1.515098,2.900787,1.262639,...,-0.302642,1.104279,-0.890172,1.039392,0.433664,-0.424274,-0.754427,-1.486754,0.211968,-0.159443
1202737,22c8381d9eab,39b04d8ac30a,15.792901,3.049137,3.662473,1.958672,-2.121026,-1.705861,0.289672,-0.145055,...,0.478689,-0.994209,1.299318,-0.486527,1.627801,-1.688751,1.098206,0.748161,0.059967,0.246059


In [ ]:
fasttext_svd42_test_filtered

,item_id,user_id,fasttext_svd_0,fasttext_svd_1,fasttext_svd_2,fasttext_svd_3,fasttext_svd_4,fasttext_svd_5,fasttext_svd_6,fasttext_svd_7,...,fasttext_svd_32,fasttext_svd_33,fasttext_svd_34,fasttext_svd_35,fasttext_svd_36,fasttext_svd_37,fasttext_svd_38,fasttext_svd_39,fasttext_svd_40,fasttext_svd_41
0,bd45a493569d,6be13830198e,14.578194,0.672830,3.752356,2.741311,-1.879419,0.717007,-1.156460,-3.143728,...,0.768782,0.097409,1.095845,2.418682,0.580718,1.037419,0.085969,0.947173,1.960517,-1.605448
1,321127a6a33b,98b93e9aaf33,13.052496,4.107162,-5.243079,1.656195,-2.957542,1.713545,1.956829,0.864946,...,0.648093,-0.233521,-0.215711,-0.096786,0.020301,-1.343766,0.600153,-0.504397,-0.285129,0.155315
2,d976d5211526,7246f2b892e9,18.410118,-4.249895,1.679796,-1.569762,-0.417480,-5.554393,-0.563870,-2.301928,...,-0.866018,-0.245479,1.024702,0.709700,0.087260,0.048417,0.699840,1.247751,-2.406474,0.317073
3,c046d8685345,3ceed1f36bec,13.266272,3.919538,-2.327638,6.627603,0.513288,1.813976,0.959125,0.097811,...,0.088975,-0.158591,-0.784938,-1.647027,0.717775,0.332870,-0.000949,-0.587898,0.106513,0.197769
4,98a8ba1ff4c4,feb28eeb11a9,17.308040,-6.411292,5.508885,-2.264929,-0.979141,3.572682,-0.892608,-0.703163,...,0.420557,0.996238,0.488861,-0.210143,-1.972112,-0.882952,0.600725,-0.250321,0.754733,0.014643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300680,07a88b2f9ffc,868bce3fbbb2,15.231983,-5.558916,0.798148,-0.743809,0.560363,-3.235836,-0.821792,-1.345079,...,-0.408185,1.396748,0.412664,-0.613132,-0.105831,0.953039,0.118109,-0.670416,-0.432826,-0.047174
300681,a7b7029023e4,3217919a362d,11.967002,1.721828,-1.660143,0.284126,-2.157236,-2.796931,-4.290546,-0.169040,...,-0.752119,0.711257,1.195708,1.419499,0.331028,1.055385,-1.086039,-0.796101,1.004129,-0.682739
300682,efecf8d66b20,cdb3b17927a8,15.588081,-3.112793,-1.764992,-0.915786,3.535912,0.731315,-2.425103,3.454228,...,0.297870,2.141428,-0.081264,0.471291,-1.437182,-3.236323,-1.753408,1.066258,0.375393,0.209052
300683,b4e141ae7703,845b032118f6,12.843407,-6.786009,-1.645597,1.324130,0.587664,0.985758,2.575457,-0.178568,...,-0.984321,0.416224,0.547521,-0.264002,0.738177,0.359382,-0.197295,-2.863683,0.809867,0.567793


In [ ]:
merged_train = tfidf_svd5_train.merge(
    fasttext_svd42_train_filtered, on=['item_id', 'user_id'], how='left'
).merge(
    train_spacy_svd40, on=['item_id', 'user_id'], how='left'
)
merged_train

,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,user_type,region_city,...,spacy_svd_30,spacy_svd_31,spacy_svd_32,spacy_svd_33,spacy_svd_34,spacy_svd_35,spacy_svd_36,spacy_svd_37,spacy_svd_38,spacy_svd_39
0,Омская область,Омск,Бытовая электроника,Настольные компьютеры,NaN,NaN,NaN,50.0,Private,Омская область_Омск,...,0.070859,-0.200402,0.399969,0.464117,-0.783324,0.042151,0.461139,0.033606,0.273473,0.345149
1,Башкортостан,Уфа,Хобби и отдых,Спорт и отдых,Зимние виды спорта,NaN,NaN,500.0,Private,Башкортостан_Уфа,...,-0.460100,0.109586,-0.204724,-0.232934,-0.253469,0.013493,0.305173,0.093155,0.241529,0.004352
2,Нижегородская область,Дзержинск,Для дома и дачи,Растения,NaN,NaN,NaN,80.0,Private,Нижегородская область_Дзержинск,...,-0.009834,-0.376687,0.270234,0.186656,-0.457154,0.029784,-0.187105,-0.593824,-0.213089,0.623080
3,Челябинская область,Миасс,Личные вещи,Детская одежда и обувь,Для девочек,Верхняя одежда,98-104 см (2-4 года),1500.0,Private,Челябинская область_Миасс,...,-0.146123,0.093856,-0.400256,-0.540356,0.629810,-0.108631,-0.443335,0.297444,0.112421,0.046438
4,Башкортостан,Уфа,Для дома и дачи,Бытовая техника,Для кухни,Плиты,NaN,18990.0,Shop,Башкортостан_Уфа,...,0.334175,0.183384,-0.140712,0.173028,-0.004446,0.184789,-0.004080,0.102282,0.066628,0.176290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202734,Ростовская область,Ростов-на-Дону,Услуги,Предложение услуг,"Ремонт, строительство",Сантехника,NaN,NaN,Private,Ростовская область_Ростов-на-Дону,...,0.480721,0.232176,-0.072481,-0.010870,0.239673,0.211701,-0.043929,0.193612,-0.128270,-0.304175
1202735,Тульская область,Тула,Недвижимость,Квартиры,Сдам,На длительный срок,2,13500.0,Company,Тульская область_Тула,...,0.241369,0.064511,0.057972,0.236461,0.251034,-0.197390,0.153372,0.239418,-0.094845,0.110294
1202736,Белгородская область,Алексеевка,Услуги,Предложение услуг,Другое,NaN,NaN,700.0,Private,Белгородская область_Алексеевка,...,-0.435914,0.283831,-0.043896,0.012694,0.239465,0.096773,0.053324,-0.124128,0.009042,-0.077851
1202737,Челябинская область,Челябинск,Личные вещи,Товары для детей и игрушки,Детские коляски,NaN,NaN,9000.0,Private,Челябинская область_Челябинск,...,0.093592,-0.526595,-0.191935,-0.310929,0.004045,-0.137020,-0.117629,-0.120800,0.071888,0.052071


In [ ]:
merged_test = tfidf_svd5_test.merge(
    fasttext_svd42_test_filtered, on=['item_id', 'user_id'], how='left'
).merge(
    test_spacy_svd40, on=['item_id', 'user_id'], how='left'
)
merged_test

,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,user_type,title,...,spacy_svd_30,spacy_svd_31,spacy_svd_32,spacy_svd_33,spacy_svd_34,spacy_svd_35,spacy_svd_36,spacy_svd_37,spacy_svd_38,spacy_svd_39
0,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Аксессуары,NaN,NaN,50000.0,Private,Сумка бренд Louis Vuitton,...,-0.322273,-0.001168,-0.379320,0.000552,-0.002261,0.064362,0.157960,-0.190988,-0.172207,0.084610
1,Ярославская область,Ярославль,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,21950.0,Company,Автокресло Carmate (Ailebebe) Swing Moon Beige,...,0.108131,0.112723,0.056348,0.078313,0.076306,-0.016968,0.022123,0.037728,0.052844,-0.016502
2,Тюменская область,Тюмень,Личные вещи,"Одежда, обувь, аксессуары",Мужская одежда,Пиджаки и костюмы,52–54 (XXL),2500.0,Private,Пиджак кожанный,...,0.105742,0.015035,0.180098,-0.219161,-0.253191,-0.230806,-0.205053,0.281821,0.577967,0.178472
3,Новосибирская область,Новосибирск,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,32,750.0,Company,Сапоги на слякоть котофей нов,...,0.007149,0.171786,0.181411,0.056841,-0.141842,-0.184129,0.066908,-0.142360,-0.032983,0.037495
4,Краснодарский край,Новороссийск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,42–44 (S),200.0,Private,Юбка,...,0.004518,-0.681913,-0.230038,-0.141053,-0.118111,0.202493,-0.027205,-0.149595,0.102804,-0.392306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300680,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Обувь,37,1000.0,Private,Сабо,...,-0.013230,-0.614357,-0.182608,-0.107221,-0.118657,-0.377602,0.230813,0.269826,-0.069822,0.159903
300681,Свердловская область,Екатеринбург,Бытовая электроника,Телефоны,LG,NaN,NaN,5500.0,Private,Lg Leon h 324,...,0.037777,0.244492,-0.216132,0.093566,-0.036233,-0.127820,0.511694,-0.208028,-0.229156,0.217439
300682,Тульская область,Тула,Личные вещи,Товары для детей и игрушки,Товары для купания,NaN,NaN,150.0,Private,Ванна детская,...,-0.100290,-0.262300,-0.080573,-0.250221,0.120220,-0.127277,0.253796,0.186099,0.152250,0.225946
300683,Свердловская область,Екатеринбург,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,24,300.0,Private,"Сапоги эва, размер 24-25",...,0.666533,0.015940,-0.294694,-0.424865,0.389159,-0.019254,0.172634,0.147752,0.118220,0.071372


In [ ]:
print(merged_train.columns)

Index(['region', 'city', 'parent_category_name', 'category_name', 'param_1',
       'param_2', 'param_3', 'price', 'user_type', 'region_city',
       ...
       'spacy_svd_30', 'spacy_svd_31', 'spacy_svd_32', 'spacy_svd_33',
       'spacy_svd_34', 'spacy_svd_35', 'spacy_svd_36', 'spacy_svd_37',
       'spacy_svd_38', 'spacy_svd_39'],
      dtype='object', length=128)


In [ ]:
merged_test

,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,user_type,region_city,...,spacy_svd_30,spacy_svd_31,spacy_svd_32,spacy_svd_33,spacy_svd_34,spacy_svd_35,spacy_svd_36,spacy_svd_37,spacy_svd_38,spacy_svd_39
0,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Аксессуары,NaN,NaN,50000.0,Private,Красноярский край_Красноярск,...,-0.322273,-0.001168,-0.379320,0.000552,-0.002261,0.064362,0.157960,-0.190988,-0.172207,0.084610
1,Ярославская область,Ярославль,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,21950.0,Company,Ярославская область_Ярославль,...,0.108131,0.112723,0.056348,0.078313,0.076306,-0.016968,0.022123,0.037728,0.052844,-0.016502
2,Тюменская область,Тюмень,Личные вещи,"Одежда, обувь, аксессуары",Мужская одежда,Пиджаки и костюмы,52–54 (XXL),2500.0,Private,Тюменская область_Тюмень,...,0.105742,0.015035,0.180098,-0.219161,-0.253191,-0.230806,-0.205053,0.281821,0.577967,0.178472
3,Новосибирская область,Новосибирск,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,32,750.0,Company,Новосибирская область_Новосибирск,...,0.007149,0.171786,0.181411,0.056841,-0.141842,-0.184129,0.066908,-0.142360,-0.032983,0.037495
4,Краснодарский край,Новороссийск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,42–44 (S),200.0,Private,Краснодарский край_Новороссийск,...,0.004518,-0.681913,-0.230038,-0.141053,-0.118111,0.202493,-0.027205,-0.149595,0.102804,-0.392306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300680,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Обувь,37,1000.0,Private,Красноярский край_Красноярск,...,-0.013230,-0.614357,-0.182608,-0.107221,-0.118657,-0.377602,0.230813,0.269826,-0.069822,0.159903
300681,Свердловская область,Екатеринбург,Бытовая электроника,Телефоны,LG,NaN,NaN,5500.0,Private,Свердловская область_Екатеринбург,...,0.037777,0.244492,-0.216132,0.093566,-0.036233,-0.127820,0.511694,-0.208028,-0.229156,0.217439
300682,Тульская область,Тула,Личные вещи,Товары для детей и игрушки,Товары для купания,NaN,NaN,150.0,Private,Тульская область_Тула,...,-0.100290,-0.262300,-0.080573,-0.250221,0.120220,-0.127277,0.253796,0.186099,0.152250,0.225946
300683,Свердловская область,Екатеринбург,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,24,300.0,Private,Свердловская область_Екатеринбург,...,0.666533,0.015940,-0.294694,-0.424865,0.389159,-0.019254,0.172634,0.147752,0.118220,0.071372


In [ ]:
merged_test = merged_test.drop(columns=['text', 'description', 'title'])

In [ ]:
merged_train.to_csv(data_dir_path+'FINAL_TabularFeature_with_3TextEmbeddings_train.csv', index=False)
merged_test.to_csv(data_dir_path+'FINAL_TabularFeature_with_3TextEmbeddings_test.csv', index=False)
